In [14]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.colors import LinearSegmentedColormap

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

In [15]:
# Read processed point cloud data

point_dictionary_path = '/home/chris/Code/PointClouds/data/hull_pcd/morten_plate/mapped_hull_points_dictionary.pkl'

hull_centers_dictionary = {}

with open(point_dictionary_path, 'rb') as f:
    hull_points_dictionary = pickle.load(f)

hull_pcd = o3d.geometry.PointCloud()

for label, points in hull_points_dictionary.items():
    hull_pcd.points.extend(o3d.utility.Vector3dVector(points))
    center = np.mean(points, axis=0)
    hull_centers_dictionary[label] = center
    
hull_pcd.paint_uniform_color([0.1, 1, 0.1])

hull_centers_pcd = o3d.geometry.PointCloud()
hull_centers_points = list(hull_centers_dictionary.values())
hull_centers_pcd.points = o3d.utility.Vector3dVector(np.array(hull_centers_points))
hull_centers_pcd.paint_uniform_color([1, 0, 0])  # Color centers red

print("Point cloud data has been loaded successfully.")

Point cloud data has been loaded successfully.


In [16]:
# Read cad model data (STEP file)

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

Step file has been loaded successfully.


In [17]:
# Extract edges at the top Z-coordinate

def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
#o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

Generating top view edges and sampling to create the associated point cloud...
Total number of top view edges: 586
Total number of points in the cad point cloud: 81774
Top view cad point cloud created.


In [18]:
# # Apply clustering to the cad_pcd

# eps = 0.50
# min_points = 10

# cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

# max_label = cad_shapes_labels.max()
# print(f"Cad point cloud has {max_label + 1} clusters")
# base_cmap = plt.get_cmap("tab20")
# color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
# colors = np.array(color_cycle)[cad_shapes_labels]
# colors[cad_shapes_labels<0] = 0
# cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

# cad_points_dictionary = {}
# cad_centers_dictionary = {}

# # Extract the points and calculate the centers for each cluster
# for label in np.unique(cad_shapes_labels):
#     if label != -1:  # Ignore noise
#         cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
#         cad_points_dictionary[label] = cluster_points
#         cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

# cad_centers_pcd = o3d.geometry.PointCloud()
# cad_centers_points = list(cad_centers_dictionary.values())
# cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
# cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

# #o3d.visualization.draw_geometries([cad_pcd])

In [19]:
#VISUALIZE HERE
o3d.visualization.draw_geometries([cad_pcd, hull_pcd])

In [ ]:
#Align the centers of the two clouds for an initial alignment

cad_pcd_center = np.mean(np.asarray(cad_pcd.points), axis=0)
hull_pcd_center = np.mean(np.asarray(hull_pcd.points), axis=0)

# Compute the translation vector
translation = cad_pcd_center - hull_pcd_center

# Translate the hull_pcd to align the centers
hull_pcd.points = o3d.utility.Vector3dVector(np.asarray(hull_pcd.points) + translation)

#o3d.visualization.draw_geometries([hull_pcd, cad_pcd])

In [ ]:
front =  [0.0, 0.0, 1.0]
lookat = [-105.36407274754953, -106.22557127184305, 2.0]
up =  [0.0, 1.0, 0.0]
zoom = 0.69999999999999996

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([target_temp, source_temp],
                                      zoom=zoom,
                                      front=front,
                                      lookat=lookat,
                                      up=up)
    
source = copy.deepcopy(hull_pcd)
target = copy.deepcopy(cad_pcd)

In [ ]:
# Total points alignment: Align point clouds through ICP based on the lowest RMSE (use all points)

def evaluate_icp(source, target, threshold, trans_init, verbose=False):
    # Perform ICP registration
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    # Evaluate the registration
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, reg_p2p.transformation)
    
    if verbose:
        print(f"Threshold: {threshold}")
        print(f"Fitness: {reg_p2p.fitness}")
        print(f"Inlier RMSE: {reg_p2p.inlier_rmse}")
        print(f"Evaluation Fitness: {evaluation.fitness}")
        print(f"Evaluation Inlier RMSE: {evaluation.inlier_rmse}")
        print("Transformation Matrix:")
        print(reg_p2p.transformation)
        print("-" * 40)
    
    return reg_p2p, evaluation.inlier_rmse, threshold

# Initial transformation (identity matrix)
trans_init = np.eye(4)

# Evaluate ICP for different thresholds
thresholds = [0.5, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.67, 0.69, 0.7, 0.71, 0.72, 0.73, 0.8, 0.85, 0.9, 0.95, 1, 1.1, 1.2, 1.25, 1.3, 1,325, 1.33, 1.34, 1.35, 1.36, 1.37, 1.375, 1.4, 1.425, 1.45, 1.475, 1.5, 2, 3]
results = [evaluate_icp(source, target, threshold, trans_init, verbose=False) for threshold in tqdm(thresholds)]
best_result = min(results, key=lambda x: x[1])

print(f'Best threshold for total alignment was {best_result[2]}')

# Visual inspection for the best threshold 
draw_registration_result(source, target, best_result[0].transformation)